# 理解父类函数调用子类函数

In [12]:
class Parent(object):
    def __init__(self, name):
        self.name = name
    
    def get_name(self):
        pass 
    
    def print_name(self):
        self.get_name()

class Son(Parent):
    def __init__(self, name='YQS'):
        super().__init__(name)
        
    def get_name(self):
        print(self.name)

In [13]:
pt = Son()
pt.print_name()

YQS


# CONV_5 特征图的每个特征点锚框的生成（9个）

In [14]:
# 基础函数
import numpy as np 

In [18]:
def _whctrs(anchor):
    """
    Return width, height, x center, and y center for an anchor (window).
    """

    w = anchor[2] - anchor[0] + 1
    h = anchor[3] - anchor[1] + 1
    x_ctr = anchor[0] + 0.5 * (w - 1)
    y_ctr = anchor[1] + 0.5 * (h - 1)
    
    return w, h, x_ctr, y_ctr

In [19]:
def _mkanchors(ws, hs, x_ctr, y_ctr):
    """
    Given a vector of widths (ws) and heights (hs) around a center
    (x_ctr, y_ctr), output a set of anchors (windows).
    """

    ws = ws[:, np.newaxis]
    hs = hs[:, np.newaxis]
    anchors = np.hstack((x_ctr - 0.5 * (ws - 1),
                       y_ctr - 0.5 * (hs - 1),
                       x_ctr + 0.5 * (ws - 1),
                       y_ctr + 0.5 * (hs - 1)))
    
    return anchors

In [20]:
def _ratio_enum(anchor, ratios):
    """
    Enumerate a set of anchors for each aspect ratio wrt an anchor.
    """

    w, h, x_ctr, y_ctr = _whctrs(anchor) # 坐标中心化，将矩形框的原点移动到矩形框的中心点
    size = w * h
    size_ratios = size / ratios
    ws = np.round(np.sqrt(size_ratios))
    hs = np.round(ws * ratios)
    anchors = _mkanchors(ws, hs, x_ctr, y_ctr)
    
    return anchors

In [21]:
def _scale_enum(anchor, scales):
    """
    Enumerate a set of anchors for each scale wrt an anchor.
    """

    w, h, x_ctr, y_ctr = _whctrs(anchor)
    ws = w * scales
    hs = h * scales
    anchors = _mkanchors(ws, hs, x_ctr, y_ctr)
    
    return anchors

In [22]:
def generate_anchors(base_size=16, ratios=[0.5, 1, 2],
                 scales=2 ** np.arange(3, 6)):
    """
    Generate anchor (reference) windows by enumerating aspect ratios X
    scales wrt a reference (0, 0, 15, 15) window.
    """
    
    base_anchor = np.array([1, 1, base_size, base_size]) - 1 # 基础框的坐标
    ratio_anchors = _ratio_enum(base_anchor, ratios)
    anchors = np.vstack([_scale_enum(ratio_anchors[i, :], scales)
                   for i in range(ratio_anchors.shape[0])])
    
    return anchors

In [24]:
base_size = 16 
anchor = np.array([1, 1, base_size, base_size]) - 1
anchor

array([ 0,  0, 15, 15])

In [26]:
w = anchor[2] - anchor[0] + 1
print(w)
h = anchor[3] - anchor[1] + 1
print(h)
x_ctr = anchor[0] + 0.5 * (w - 1)
print(x_ctr)
y_ctr = anchor[1] + 0.5 * (h - 1)
print(y_ctr)

16
16
7.5
7.5


In [48]:
size = w * h
print(size)
ratios=[0.5, 1, 2] # 按面积计算比例
size_ratios = size / ratios
print(size_ratios)
print(np.sqrt(size_ratios))
ws = np.round(np.sqrt(size_ratios))
print(ws)
hs = np.round(ws * ratios)
print(hs)

256
[512. 256. 128.]
[22.627417  16.        11.3137085]
[23. 16. 11.]
[12. 16. 22.]


In [49]:
# 三种比例锚框的宽
ws = ws[:, np.newaxis]
print(ws)
# 三种比例锚框的高
hs = hs[:, np.newaxis]
print(hs)
print(x_ctr - 0.5 * (ws - 1))

[[23.]
 [16.]
 [11.]]
[[12.]
 [16.]
 [22.]]
[[-3.5]
 [ 0. ]
 [ 2.5]]


In [43]:
np.hstack?

In [50]:
# 图像是一个第四象限的坐标系，向下y+,向右x+
# 矩形框的表示为（左上，右下）两个点坐标
# np.hstack（）在水平方向上平铺
# 获取三种比例锚框
anchors = np.hstack((x_ctr - 0.5 * (ws - 1),
                   y_ctr - 0.5 * (hs - 1),
                   x_ctr + 0.5 * (ws - 1),
                    y_ctr + 0.5 * (hs - 1)))
print(anchors)

[[-3.5  2.  18.5 13. ]
 [ 0.   0.  15.  15. ]
 [ 2.5 -3.  12.5 18. ]]


In [51]:
# anchors = np.vstack([_scale_enum(ratio_anchors[i, :], scales)
#                    for i in range(ratio_anchors.shape[0])])
scales=2 ** np.arange(3, 6)
print(scales)

[ 8 16 32]


In [52]:
w, h, x_ctr, y_ctr = _whctrs(anchors[0, :])
print(w, h, x_ctr, y_ctr)

23.0 12.0 7.5 7.5


In [53]:
ws = w * scales
print(ws)
hs = h * scales
print(hs)

[184. 368. 736.]
[ 96. 192. 384.]


In [54]:
anchors_ = _mkanchors(ws, hs, x_ctr, y_ctr)
print(anchors_)

[[ -84.  -40.   99.   55.]
 [-176.  -88.  191.  103.]
 [-360. -184.  375.  199.]]


In [55]:
print(np.vstack(([1,2,3],[3,4,3])))

[[1 2 3]
 [3 4 3]]


In [57]:
anchors__ = np.vstack([_scale_enum(anchors[i, :], scales)
                   for i in range(anchors.shape[0])])
anchors__

array([[ -84.,  -40.,   99.,   55.],
       [-176.,  -88.,  191.,  103.],
       [-360., -184.,  375.,  199.],
       [ -56.,  -56.,   71.,   71.],
       [-120., -120.,  135.,  135.],
       [-248., -248.,  263.,  263.],
       [ -36.,  -80.,   51.,   95.],
       [ -80., -168.,   95.,  183.],
       [-168., -344.,  183.,  359.]])

In [58]:
np.vstack?

In [60]:
import tensorflow as tf 

In [61]:
# 为每个点生成9个锚框的函数
def generate_anchors_pre_tf(height, width, feat_stride=16, anchor_scales=(8, 16, 32), anchor_ratios=(0.5, 1, 2)):
    shift_x = tf.range(width) * feat_stride # width
    shift_y = tf.range(height) * feat_stride # height
    shift_x, shift_y = tf.meshgrid(shift_x, shift_y)
    sx = tf.reshape(shift_x, shape=(-1,))
    sy = tf.reshape(shift_y, shape=(-1,))
    shifts = tf.transpose(tf.stack([sx, sy, sx, sy]))
    K = tf.multiply(width, height)
    shifts = tf.transpose(tf.reshape(shifts, shape=[1, K, 4]), perm=(1, 0, 2))

    anchors = generate_anchors(ratios=np.array(anchor_ratios), scales=np.array(anchor_scales))
    A = anchors.shape[0]
    anchor_constant = tf.constant(anchors.reshape((1, A, 4)), dtype=tf.int32)

    length = K * A
    anchors_tf = tf.reshape(tf.add(anchor_constant, shifts), shape=(length, 4))

    return tf.cast(anchors_tf, dtype=tf.float32), length

In [62]:
height, width = 10, 10 
feat_stride = 16 
anchor_scales=(8, 16, 32)
anchor_ratios=(0.5, 1, 2)

In [68]:
shift_x = tf.range(width) * feat_stride # width
print(shift_x)
shift_y = tf.range(height) * feat_stride # height
print(shift_y)

tf.Tensor([  0  16  32  48  64  80  96 112 128 144], shape=(10,), dtype=int32)
tf.Tensor([  0  16  32  48  64  80  96 112 128 144], shape=(10,), dtype=int32)


In [69]:
# 获取特征图每个点的坐标
shift_x, shift_y = tf.meshgrid(shift_x, shift_y)
print(shift_x, shift_y)

tf.Tensor(
[[  0  16  32  48  64  80  96 112 128 144]
 [  0  16  32  48  64  80  96 112 128 144]
 [  0  16  32  48  64  80  96 112 128 144]
 [  0  16  32  48  64  80  96 112 128 144]
 [  0  16  32  48  64  80  96 112 128 144]
 [  0  16  32  48  64  80  96 112 128 144]
 [  0  16  32  48  64  80  96 112 128 144]
 [  0  16  32  48  64  80  96 112 128 144]
 [  0  16  32  48  64  80  96 112 128 144]
 [  0  16  32  48  64  80  96 112 128 144]], shape=(10, 10), dtype=int32) tf.Tensor(
[[  0   0   0   0   0   0   0   0   0   0]
 [ 16  16  16  16  16  16  16  16  16  16]
 [ 32  32  32  32  32  32  32  32  32  32]
 [ 48  48  48  48  48  48  48  48  48  48]
 [ 64  64  64  64  64  64  64  64  64  64]
 [ 80  80  80  80  80  80  80  80  80  80]
 [ 96  96  96  96  96  96  96  96  96  96]
 [112 112 112 112 112 112 112 112 112 112]
 [128 128 128 128 128 128 128 128 128 128]
 [144 144 144 144 144 144 144 144 144 144]], shape=(10, 10), dtype=int32)


In [71]:
# 转变为一维变量
sx = tf.reshape(shift_x, shape=(-1,))
print(sx)
sy = tf.reshape(shift_y, shape=(-1,))
print(sy)

tf.Tensor(
[  0  16  32  48  64  80  96 112 128 144   0  16  32  48  64  80  96 112
 128 144   0  16  32  48  64  80  96 112 128 144   0  16  32  48  64  80
  96 112 128 144   0  16  32  48  64  80  96 112 128 144   0  16  32  48
  64  80  96 112 128 144   0  16  32  48  64  80  96 112 128 144   0  16
  32  48  64  80  96 112 128 144   0  16  32  48  64  80  96 112 128 144
   0  16  32  48  64  80  96 112 128 144], shape=(100,), dtype=int32)
tf.Tensor(
[  0   0   0   0   0   0   0   0   0   0  16  16  16  16  16  16  16  16
  16  16  32  32  32  32  32  32  32  32  32  32  48  48  48  48  48  48
  48  48  48  48  64  64  64  64  64  64  64  64  64  64  80  80  80  80
  80  80  80  80  80  80  96  96  96  96  96  96  96  96  96  96 112 112
 112 112 112 112 112 112 112 112 128 128 128 128 128 128 128 128 128 128
 144 144 144 144 144 144 144 144 144 144], shape=(100,), dtype=int32)


In [72]:
print(tf.stack([sx, sy, sx, sy]))

tf.Tensor(
[[  0  16  32  48  64  80  96 112 128 144   0  16  32  48  64  80  96 112
  128 144   0  16  32  48  64  80  96 112 128 144   0  16  32  48  64  80
   96 112 128 144   0  16  32  48  64  80  96 112 128 144   0  16  32  48
   64  80  96 112 128 144   0  16  32  48  64  80  96 112 128 144   0  16
   32  48  64  80  96 112 128 144   0  16  32  48  64  80  96 112 128 144
    0  16  32  48  64  80  96 112 128 144]
 [  0   0   0   0   0   0   0   0   0   0  16  16  16  16  16  16  16  16
   16  16  32  32  32  32  32  32  32  32  32  32  48  48  48  48  48  48
   48  48  48  48  64  64  64  64  64  64  64  64  64  64  80  80  80  80
   80  80  80  80  80  80  96  96  96  96  96  96  96  96  96  96 112 112
  112 112 112 112 112 112 112 112 128 128 128 128 128 128 128 128 128 128
  144 144 144 144 144 144 144 144 144 144]
 [  0  16  32  48  64  80  96 112 128 144   0  16  32  48  64  80  96 112
  128 144   0  16  32  48  64  80  96 112 128 144   0  16  32  48  64  80
   96 112 128 1

In [73]:
# 获取每个特征点的坐标，变为double份
shifts = tf.transpose(tf.stack([sx, sy, sx, sy]))
print(shifts)

tf.Tensor(
[[  0   0   0   0]
 [ 16   0  16   0]
 [ 32   0  32   0]
 [ 48   0  48   0]
 [ 64   0  64   0]
 [ 80   0  80   0]
 [ 96   0  96   0]
 [112   0 112   0]
 [128   0 128   0]
 [144   0 144   0]
 [  0  16   0  16]
 [ 16  16  16  16]
 [ 32  16  32  16]
 [ 48  16  48  16]
 [ 64  16  64  16]
 [ 80  16  80  16]
 [ 96  16  96  16]
 [112  16 112  16]
 [128  16 128  16]
 [144  16 144  16]
 [  0  32   0  32]
 [ 16  32  16  32]
 [ 32  32  32  32]
 [ 48  32  48  32]
 [ 64  32  64  32]
 [ 80  32  80  32]
 [ 96  32  96  32]
 [112  32 112  32]
 [128  32 128  32]
 [144  32 144  32]
 [  0  48   0  48]
 [ 16  48  16  48]
 [ 32  48  32  48]
 [ 48  48  48  48]
 [ 64  48  64  48]
 [ 80  48  80  48]
 [ 96  48  96  48]
 [112  48 112  48]
 [128  48 128  48]
 [144  48 144  48]
 [  0  64   0  64]
 [ 16  64  16  64]
 [ 32  64  32  64]
 [ 48  64  48  64]
 [ 64  64  64  64]
 [ 80  64  80  64]
 [ 96  64  96  64]
 [112  64 112  64]
 [128  64 128  64]
 [144  64 144  64]
 [  0  80   0  80]
 [ 16  80  16  80]
 

In [74]:
K = tf.multiply(width, height)
print(K)
print(tf.reshape(shifts, shape=[1, K, 4]))
shifts = tf.transpose(tf.reshape(shifts, shape=[1, K, 4]), perm=(1, 0, 2))
print(shifts)

tf.Tensor(100, shape=(), dtype=int32)
tf.Tensor(
[[[  0   0   0   0]
  [ 16   0  16   0]
  [ 32   0  32   0]
  [ 48   0  48   0]
  [ 64   0  64   0]
  [ 80   0  80   0]
  [ 96   0  96   0]
  [112   0 112   0]
  [128   0 128   0]
  [144   0 144   0]
  [  0  16   0  16]
  [ 16  16  16  16]
  [ 32  16  32  16]
  [ 48  16  48  16]
  [ 64  16  64  16]
  [ 80  16  80  16]
  [ 96  16  96  16]
  [112  16 112  16]
  [128  16 128  16]
  [144  16 144  16]
  [  0  32   0  32]
  [ 16  32  16  32]
  [ 32  32  32  32]
  [ 48  32  48  32]
  [ 64  32  64  32]
  [ 80  32  80  32]
  [ 96  32  96  32]
  [112  32 112  32]
  [128  32 128  32]
  [144  32 144  32]
  [  0  48   0  48]
  [ 16  48  16  48]
  [ 32  48  32  48]
  [ 48  48  48  48]
  [ 64  48  64  48]
  [ 80  48  80  48]
  [ 96  48  96  48]
  [112  48 112  48]
  [128  48 128  48]
  [144  48 144  48]
  [  0  64   0  64]
  [ 16  64  16  64]
  [ 32  64  32  64]
  [ 48  64  48  64]
  [ 64  64  64  64]
  [ 80  64  80  64]
  [ 96  64  96  64]
  [112  64 

In [75]:
anchors = generate_anchors(ratios=np.array(anchor_ratios), scales=np.array(anchor_scales))
print(anchors)
A = anchors.shape[0]
print(A)

[[ -84.  -40.   99.   55.]
 [-176.  -88.  191.  103.]
 [-360. -184.  375.  199.]
 [ -56.  -56.   71.   71.]
 [-120. -120.  135.  135.]
 [-248. -248.  263.  263.]
 [ -36.  -80.   51.   95.]
 [ -80. -168.   95.  183.]
 [-168. -344.  183.  359.]]
9


In [76]:
print(anchors.reshape((1, A, 4)))
# 将变量转为tensor
anchor_constant = tf.constant(anchors.reshape((1, A, 4)), dtype=tf.int32)
print(anchor_constant)

[[[ -84.  -40.   99.   55.]
  [-176.  -88.  191.  103.]
  [-360. -184.  375.  199.]
  [ -56.  -56.   71.   71.]
  [-120. -120.  135.  135.]
  [-248. -248.  263.  263.]
  [ -36.  -80.   51.   95.]
  [ -80. -168.   95.  183.]
  [-168. -344.  183.  359.]]]
tf.Tensor(
[[[ -84  -40   99   55]
  [-176  -88  191  103]
  [-360 -184  375  199]
  [ -56  -56   71   71]
  [-120 -120  135  135]
  [-248 -248  263  263]
  [ -36  -80   51   95]
  [ -80 -168   95  183]
  [-168 -344  183  359]]], shape=(1, 9, 4), dtype=int32)


In [77]:
length = K * A
print(length)
print(tf.add(anchor_constant, shifts))
# 对锚框的坐标进行平移
anchors_tf = tf.reshape(tf.add(anchor_constant, shifts), shape=(length, 4))
print(anchors_tf)

tf.Tensor(900, shape=(), dtype=int32)
tf.Tensor(
[[[ -84  -40   99   55]
  [-176  -88  191  103]
  [-360 -184  375  199]
  ...
  [ -36  -80   51   95]
  [ -80 -168   95  183]
  [-168 -344  183  359]]

 [[ -68  -40  115   55]
  [-160  -88  207  103]
  [-344 -184  391  199]
  ...
  [ -20  -80   67   95]
  [ -64 -168  111  183]
  [-152 -344  199  359]]

 [[ -52  -40  131   55]
  [-144  -88  223  103]
  [-328 -184  407  199]
  ...
  [  -4  -80   83   95]
  [ -48 -168  127  183]
  [-136 -344  215  359]]

 ...

 [[  28  104  211  199]
  [ -64   56  303  247]
  [-248  -40  487  343]
  ...
  [  76   64  163  239]
  [  32  -24  207  327]
  [ -56 -200  295  503]]

 [[  44  104  227  199]
  [ -48   56  319  247]
  [-232  -40  503  343]
  ...
  [  92   64  179  239]
  [  48  -24  223  327]
  [ -40 -200  311  503]]

 [[  60  104  243  199]
  [ -32   56  335  247]
  [-216  -40  519  343]
  ...
  [ 108   64  195  239]
  [  64  -24  239  327]
  [ -24 -200  327  503]]], shape=(100, 9, 4), dtype=int32)


In [78]:
# 数据类型转换
print(tf.cast(anchors_tf, dtype=tf.float32))

tf.Tensor(
[[ -84.  -40.   99.   55.]
 [-176.  -88.  191.  103.]
 [-360. -184.  375.  199.]
 ...
 [ 108.   64.  195.  239.]
 [  64.  -24.  239.  327.]
 [ -24. -200.  327.  503.]], shape=(900, 4), dtype=float32)
